## 一、Tomcat优化思路梳理
* 无限制的增大业务线程。
* 优化请求队列的数量。
* 第一，根据服务器情况，来制定业务线程池的数量和请求队列的数量；第二，如果业务线程和请求队列一定的情况下，让我们的业务线程处理的更快；第三，优化线程，思路:a.加大它的资源（内存更多或CPU更多），b.对线程模型的优化。

## Connector配置建议
### 线程池优化
* `maxConnections`：最大连接数。
    1. 最大连接数是受系统内核的影响。`Linux`的系统内核数，可以通过`ulimit －a`的方式查看服务器上允许的最大连接数。修改`/etc/security/limits.conf`下，把`core`和`rss`两项进行放开，把这两个都变成`nofiles`，且对应的值都是**65535**。
    2. 添加maxConnections的配置。（配置完成以后，一定要进行压测！超过Tomcat最大承载能力的时候，性能会出现急剧降低的现象。）
        2.1 **对CPU要求更高时**，建议不要配置过于大；
        2.2 **对CPU要求不是特别高时**，建议配置在**3000左右**。
    3. 配置的地方:`server.xml`中找`connector`。
* `maxThreads`：最大线程数。服务器上**500－700是相对合理的数量**。配置的位置也是在`server.xml中的connector`里面进行配置。
* `acceptCount`:最大排队等待数（也就是前面我们提到的请求队列）。**（单Tomcat能处理的最大数量就是`maxThreads`与`acceptCount`的数量之和）**。`一般建议与最大线程数持平或略低就可以接受。`

### Tomcat内存优化（能使每一个连接处理的速度更快）
JVM优化建议

参数|参数作用|优化建议
:-:|:-:|:-:
-server|启用Serveer|服务端建议开启
－Xms|最小内存|建议与－Xmx相同
－Xmx|最大内存|建议到可用内存的80%
－XX:MetaspaceSize|元空间初始值|
-XX:MaxMetaspaceSize|元空间最大内存|默认无限
-XX:MaxNewSize|新生代最大内存|默认16M

* JVM在最大内存和最小内存进行漂移的过程中，本身是浪费资源的。当最大内存和最小内存配置一样的时候，JVM就不会来回漂移。
* 上面的第4和第5是专门针对1.8及以上的。（以前是持久内存）
* 修改的位置在`tomcat/bin/catalina.sh`。通过添加`JAVA_OPTS="-server -Xms128m -Xmx128m -XX:MetaspaceSize=128m -XX:MaxMetaspaceSize=128m -XX:MaxNewSize=32m"`
* 验证的方式是通过启动后，通过`jmap -heap [进程编号]`的命令进行查看。

### Tomcat其他内容优化
* Gzip相关配置(**修改的位置在conf/server.xml中的Connector上**)
    * `compression`:设置开启Gzip压缩(True或者False)。
    * `compressableMimeType`:压缩类型()。
    * `compressMinSize`:压缩后输出内容大小(2048等具体的值)。
* 其他项配置
    * `enableLookups`:开启反查域名(某些场景下比较好用，但是一般还是不用开启)。
    * `connectionTimeout`:网络连接超时阈值。
    * `minSpareThreads`:最小空闲线程数(**没有访问的情况下保持多少活跃**)。

### Tomcat三种线程模式介绍
1. **BIO：**最稳定最老的一个连接器，使用阻塞形式处理Request请求。
2. **NIO：**使用后Java的异步IO技术，进行非阻塞形式处理Request请求。
3. **APR：**原生C语音编写的非阻塞I/O，目前性能最理想。

### Apr安装(3个内容都要下载，除此之外，还需要通过yum安装`expat`和`expat-devel`两个依赖源。)
#### apr安装步骤：
1. apr：解压至安装目录；`./configure -prefix=/usr/local/apr[安装的目录]`；`make`；`make install`；
2. apr-iconv：解压至安装目录；`./configure -prefix=/usr/local/apr-iconv[自己的安装目录] --with-apr=/usr/local/apr[apr的安装目录]`;`make`;`make install`。
3. apr-util：解压至安装目录；`./config -prefix=/usr/local/apr-util[自己的安装目录] --with-apr=/usr/local/apr[apr的安装目录] --with-apriconv=/usr/local/apr-iconv[apr-iconv的安装目录]`;`make`;`make install`。

### Tomcat的apr配置
1. 到`Tomcat`下的`bin`目录下
2. 解压`tomcat-native.tar.gz`文件，然后进入当前解压生成的目录
3. 然后进入`native`目录进行安装，通过`./config --with-apr=/usr/local/apr[自己的apr的安装目录]`。最后通过`make&make install`进行安装。
4. 修改`catalina.sh`，在其中的`JAVA_OPTS`那一行的下面添加`LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/apr/lib export LD_LIBRARY_PATH`就把apr的影响范围限制到比较小的范围。
5. 修改`server.xml`文件。修改`Connector`中的`protocl`这个协议。

### Tomcat集群方式：
1. Apache Tomcat Clustering(需要验证session等内容，做session的同步和传输，对比耗时，网络传输耗时)；
    1.1 去Tomcat的官网，找到`Clustering`内容的部分，找到相关需要配置的内容；
    1.2 去修改`server.xml`文件，在`Engine`这个节点下进行内容的添加。（如果需要修改的话，也就是改改`address`和`port`标签的内容，因为实际上是内网的交互，实际上不改也没问题。）
    1.3 对于那些需要在集群中同步数据的项目，需要在`web.xml`文件中，添加`<distributable/>`标签(这样就能解决集群中绝大多数的问题)。
2. JWT等类似机制(实际上是无状态的)；
3. MSM(Memcached-Session-Manager，把session交给`Memcached`进行管理，这种方式的特点是`Memcached`在内存的利用方面无人能及，**比redis性能提升3%-5%**)、Tomcat+Redis(**具有持久化，和多种多样的数据形式**)等统一Session管理；

> 我们可以通过安装一个小软件来实现文件的上传和下载。这个是安装在`centos`系统中的，可以通过`yum -y install lrzsz`来进行安装。之后可以直接通过`rz`命令进行上传。(**但是在网络服务器中，这种并不好使！！**)